In [5]:
from ngram import ngram
import os
import preprocess
from gt_ngram import gt_ngram

indir_pre = os.getcwd() + "/"
outdir_pre = os.getcwd() + "/"
topics = ['atheism', 'autos', 'graphics', 'medicine', 'motorcycles', 'religion', 'space']

def random_sentence_ngram(n = 2, sent_pre = "I have", topics = ["autos"]):
    # TODO: lili write print like first line in ompare_perplexity_ngram()
    for topic in topics:

        indir = indir_pre + "data/classification_task/{}/train_docs".format(topic)
        content = preprocess.preprocess_dir(indir)
        ngrams = ngram(content)
        print "\n\n\nTopic: {}\n".format(topic)
        for k in xrange(1, n + 1):
            print "[{}-gram]\n".format(k)

            print "Empty sentence"
            for i in xrange(3):
                print "[{}]  ".format(i + 1) + ngrams.generate_sentence(k)

            print "\nWith incomplete sentence: " + "\"{}\"".format(sent_pre)
            for i in xrange(3):
                print "[{}]  ".format(i + 1) + ngrams.generate_sentence(k, sent_pre)


def topic_classification():
    pass


def spell_checker():
    pass


def compare_perplexity_ngram():
    print "[task]: compare perplexity for different n for good turing ngram model"

    indir = indir_pre + "data/classification_task/atheism/train_docs"
    test_f = indir_pre + "data/classification_task/test_for_classification/file_0.txt"

    content = preprocess.preprocess_dir(indir)
    atheism = gt_ngram(content)

    sentences = preprocess.preprocess_file(test_f)
    print atheism.generate_perplexity(1, sentences)
    print atheism.generate_perplexity(2, sentences)
    print atheism.generate_perplexity(3, sentences)
    print atheism.generate_perplexity(5, sentences)
    print atheism.generate_perplexity(6, sentences)
    print atheism.generate_perplexity(7, sentences)


def main():
    random_sentence_ngram(topics=topics)
    compare_perplexity_ngram()



if __name__ == "__main__":
    main()





Topic: atheism

[1-gram]

Empty sentence
[1]  To be
[2]  , . the thousand only really question his . ridicule what out faith firmly the the lack !
[3]  , wide but fundamentalist of discouraged that latter , . a is must that

With incomplete sentence: "I have"
[1]  I have the me
[2]  I have you for
[3]  I have decision achieve people worth been the the article question not by is evacuated jon a hearing the or general understood , have emotion follow and I with but how find made same a . official around enemies damage here sacred made given on increase the he only said , in person the . a beliefs situation say an is computer some d you comment it you rise tolerance wives a in in though words . history society described and hebrew get the made makes not my in were bit , of for that
[2-gram]

Empty sentence
[1]  These people to nazis ?
[2]  Given no hay mas realidad , but my saying that we do so there would be lost the language problem with the victim ?
[3]  It was issued in the words ,